In [97]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

In [98]:
# 生成数据集
def synthetic_data(w, b, num_examples): 
    '''生成y=Xw+b+噪声'''
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

In [99]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))  # 下标
    random.shuffle(indices)  # 将下标打乱
    for i in range(0, num_examples, batch_size): 
        batch_indices = torch.tensor(
            indices[i:min(i + batch_size, num_examples)])  # 从indices数组中取下标，每次取batch_size个
        yield features[batch_indices], labels[batch_indices]  # 将batch_size这批数据返回


In [100]:
# 定义初始化模型参数
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [101]:
# 定义模型
def linreg(X, w, b):
    return torch.matmul(X, w) + b

In [102]:
# 定义损失函数
def squared_loss(y_hat, y):
    '''均方损失'''
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

In [103]:
# 定义优化算法
def sgd(params, lr, batch_size):
    """小批量随机梯度下降"""
    with torch.no_grad(): # 不希望在计算过程中产生梯度
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_() # 清零梯度

In [107]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

batch_size = 10
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)  # 数据集

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels): # 每次从数据集中取batch_size个
        l = loss(net(X, w, b), y) # 计算损失函数
        l.sum().backward() # 损失函数对w, b求导
        sgd([w, b], lr, batch_size)  # 分别对w, b执行梯度下降
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch{epoch + 1}, loss {float(train_l.mean()):f}')

        

epoch1, loss 0.000048
epoch2, loss 0.000048
epoch3, loss 0.000048


In [105]:
print(f'w的估计误差:{true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差:{true_b - b}')

w的估计误差:tensor([-0.0008,  0.0004], grad_fn=<SubBackward0>)
b的估计误差:tensor([0.0007], grad_fn=<RsubBackward1>)
